In [1]:
import cadquery as cq

In [2]:
from jupyter_cadquery.occ import show, Part, Assembly

In [18]:
import html

def rgb(assy):
    def b(x):
        return int(255*x)
    
    if assy.color is None:
        return "#aaa"
    rgb = assy.color.wrapped.GetRGB()
    return "#%02x%02x%02x" % (b(rgb.Red()), b(rgb.Green()), b(rgb.Blue()))

def convert(assy, loc=None):
    loc = assy.loc if loc is None else loc * assy.loc
    color = rgb(assy)
    parent = [Part(shape.located(loc).wrapped, "%s_%d" % (assy.name, i), color=color) for i, shape in enumerate(assy.shapes)]
    children = [convert(c, loc) for c in assy.children]
    return Assembly(parent + children, assy.name)

def show_constraint(assy, q1, q2):
    parts = []
    for q in (q1, q2):
        name, kind, arg = q.split("@")
        obj = assy.objects[name].obj
        parts += [
            Part(obj.objects[0].located(loc).wrapped, name=name, show_faces=False), 
            Part(assy._query(q)[1].wrapped, name=html.escape(q))
        ]
    show(Assembly(parts))

In [10]:
def L(x,y,z):
    return cq.Location(cq.Vector(x, y, z))
def C(c):
    return cq.Color(c)

In [12]:
b1 = cq.Workplane().box(1, 1, 2).faces('>Z').edges('>X').chamfer(0.4)
b2 = cq.Workplane().box(.1, 2, 1).faces('>Z').edges('>Y').chamfer(0.1)
b3 = cq.Workplane().box(2, .1, .5).faces('>Z').edges('>X').chamfer(0.1)
b4 = cq.Workplane().box(1, 1, .2).faces('>Y').edges('>X').chamfer(0.1)

assy = cq.Assembly(b1, loc=L(1, 1, 0), name="TOP")
assy2 = cq.Assembly(b2, loc=L(0, -3, 4), name="SECOND")
assy3 = cq.Assembly(b3, loc=L(0, 0, 5), name="THIRD", color=C('orange'))

assy.add(assy2, color=C("green"))
assy.add(assy3)
assy.add(b4, loc=L(0, 0, 5), name="4th",color=C("blue1"))

In [19]:
show(convert(assy))

In [25]:
assy.constrain("TOP@faces@>(1,0,1)", "THIRD@faces@>Z", "Plane")
assy.constrain("TOP@faces@<Y", "SECOND@faces@<Y", "Axis")
assy.constrain("THIRD@faces@<X", "SECOND@faces@>X", "Plane")
#assy.constrain("SECOND@faces@>Z", "THIRD@faces@<Z", "Axis")
assy.constrain("4th@faces@>Z", "SECOND@faces@<Z", "Plane")
assy.constrain("4th@faces@>Y", "TOP@faces@<Y", "Axis")
assy.solve()

b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'


In [30]:
show(convert(assy))

In [20]:
show_constraint(assy, "TOP@faces@>(1,0,1)", "THIRD@faces@>Z")

In [103]:
show_constraint(assy, "TOP@faces@<Y", "SECOND@faces@<Y")

In [105]:
show_constraint(assy, "SECOND@faces@>Z", "THIRD@faces@<Z")

In [106]:
show_constraint(assy, "THIRD@faces@<X", "SECOND@faces@>X")

In [107]:
show_constraint(assy, "4th@faces@>Z", "SECOND@faces@<Z")

In [108]:
show_constraint(assy, "4th@faces@>Y", "TOP@faces@<Y")